<a href="https://colab.research.google.com/github/jotagectti/Natural-Language-Processing-PyTorch/blob/main/4(2)_Multi_class_text_classif_using_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Designing RNN for MulticlassTextClassification

In [37]:
import torch

In [40]:
def letterToTensor(letter):
  tensor = torch.zeros(1,n_letters)
  tensor[0][all_letters.find(letter)] = 1

  return tensor

In [41]:
print(letterToTensor('a'))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [43]:
print(letterToTensor('z'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [44]:
def nameToTensor(name):
  tensor = torch.zeros(len(name),1,n_letters)

  for li, letter in enumerate(name):
    tensor[li][0][all_letters.find(letter)] = 1
  
  return tensor

In [45]:
jorge_tensor = nameToTensor('Jorge')
jorge_tensor.size()

torch.Size([5, 1, 57])

In [46]:
jorge_tensor

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0

In [47]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
      return torch.zeros(1,self.hidden_size)

In [50]:
n_hidden = 256

rnn = RNN(n_letters, n_hidden, n_languages)

In [51]:
inp = letterToTensor('C')

hidden = torch.zeros(1,n_hidden)

output, next_hidden = rnn(inp,hidden)

print('output size: ', output.size())
print('next_hidden size: ', next_hidden.size())


output size:  torch.Size([1, 18])
next_hidden size:  torch.Size([1, 256])


In [52]:
inp = nameToTensor('Panchita')

hidden = torch.zeros(1,n_hidden)

output, next_hidden = rnn(inp[0],hidden)

In [53]:
print(output)

tensor([[-2.9173, -2.8852, -2.9347, -2.8363, -2.8196, -2.9640, -2.8862, -2.7940,
         -2.9046, -2.9705, -2.8591, -2.9510, -2.8706, -2.9570, -2.8770, -2.8987,
         -2.8218, -2.9022]], grad_fn=<LogSoftmaxBackward>)


In [54]:
def languageFromOutput(output):
  _, top_i = output.topk(1)
  language_i = top_i[0].item()
  return all_languages[language_i], language_i

In [55]:
print(languageFromOutput(output))

('Arabic', 7)


In [56]:
import random
def randomTrainingExample():
  random_language_index = random.randint(0,n_languages-1)
  language = all_languages[random_language_index]

  random_language_names = language_names[language]

  name = random_language_names[random.randint(0,len(random_language_names)-1)]

  language_tensor = torch.tensor([all_languages.index(language)], dtype=torch.long)
  name_tensor = nameToTensor(name)

  return language, name, language_tensor, name_tensor

In [57]:
for i in range(10):
  language, name, language_tensor, name_tensor = randomTrainingExample()
  print('language=', language, 'name = ',name)

language= Korean name =  Cho
language= Irish name =  Dalach
language= English name =  Waring
language= Greek name =  Pantelakos
language= Arabic name =  Wasem
language= Chinese name =  Lao
language= Russian name =  Jupanenko
language= Dutch name =  Alphen
language= Greek name =  Mentis
language= Scottish name =  Docherty
